# Load All Data

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### Load Electricity Cost

In [ ]:
from glob import glob
import pandas as pd

electricity_cost_df = pd.DataFrame()
electricity_cost_data_paths = glob("../data/clean/2*.xls")
for electricity_cost_data_path in electricity_cost_data_paths:
    electricity_cost_df = pd.concat([pd.read_csv(electricity_cost_data_path), electricity_cost_df])
electricity_cost_df["Datetime"] = pd.to_datetime(electricity_cost_df["Datetime"], format="%d-%b-%y")

In [ ]:
electricity_cost_df.sort_values("Datetime").head()

### Load Energy Usage

In [ ]:
energy_df = pd.read_csv("../data/clean/LondonEnergy.xls")
energy_df["Date"] = pd.to_datetime(energy_df["Date"], format="%m/%d/%Y")
total_starting_rows = len(energy_df.index)
energy_df.sort_values("Date").head()

### Load Weather Data

In [ ]:
weather_df = pd.read_csv("../data/clean/LondonWeather.xls")
weather_df["date"] = pd.to_datetime(weather_df["date"], format="%Y%m%d")
weather_df.sort_values("date").head()

# Merge Datasets

In [ ]:
merged_df = pd.merge(energy_df, electricity_cost_df, left_on="Date", right_on="Datetime")
merged_df = pd.merge(merged_df, weather_df, left_on="Date", right_on="date")
print(merged_df.columns)
merged_df.head()

In [ ]:
assert(len(merged_df.index) == total_starting_rows) # Assert we didn't lose any data

In [ ]:
# Drop redundant date columns
merged_df = merged_df.drop(["Datetime", "date"], axis=1)

# Test Train Split
For the test train split on time series data, we will use a rolling k fold

https://stats.stackexchange.com/questions/14099/using-k-fold-cross-validation-for-time-series-model-selection
```
Split 1: Test [1] Train [2]
Split 2: Test [1, 2] Train [3]
Split 3: Test [1, 2, 3] Train [4]
Split 4: Test [1, 2, 3, 4] Train [5]
Split 5: Test [1, 2, 3, 4, 5] Train [6]
```

In [ ]:
import numpy as np

first_date = merged_df["Date"].min()
merged_df['DateIndex'] = merged_df["Date"].apply(lambda x: (x- first_date).days).astype(int)
merged_df = merged_df.reindex(np.arange(len(merged_df.index)))
merged_df = merged_df.sort_values("DateIndex")
merged_df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.title("Data Distrubtion by Date")
sns.histplot(merged_df["Date"], bins=92)

In [ ]:
# Split Data into 6 even segments.
SPLIT_COUNT = 6

splits = []
date_indexes = merged_df["DateIndex"]
data_length = len(date_indexes)
for i in range(SPLIT_COUNT):
    split = date_indexes[int(i * data_length / SPLIT_COUNT):int((i+1) * data_length / SPLIT_COUNT)]
    splits.append((split.min(), split.max()))
    if i != 0:
        if splits[i][0] == splits[i-1][1]:
            splits[i] = (splits[i][0] + 1, splits[i][1])

splits

In [ ]:
merged_df["split"] = -1
for i, split in enumerate(splits):
    print(f"Split {i} has length: {len(merged_df[(merged_df['DateIndex'] >= split[0]) & (merged_df['DateIndex'] <= split[1])].index )}")
    merged_df.loc[(merged_df["DateIndex"] >= split[0]) & (merged_df["DateIndex"] <= split[1]), "split"] = i

In [ ]:
plt.figure(figsize=(10,10))
plt.title("6 Folds for Test Train Split")
sns.histplot(data=merged_df, x="DateIndex", hue="split", multiple="stack", bins=92)

In [ ]:
import os
os.makedirs("../data/test_train_split", exist_ok=True)

for i in range(len(splits) - 1):
    train_df = merged_df[merged_df["split"] <= i]
    train_df = train_df.drop(["DateIndex", "split"], axis=1)
    print(f"Train {i} has length {len(train_df.index)}")
    train_df.to_csv(f"../data/test_train_split/train_{i}.csv", index=False)

    test_df = merged_df[merged_df["split"] == i+1]
    test_df = test_df.drop(["DateIndex", "split"], axis=1)
    print(f"Test {i} has length {len(test_df.index)}")
    test_df.to_csv(f"../data/test_train_split/test_{i}.csv", index=False)